In [154]:
import pandas as pd
from collections import defaultdict
import json, math, gdown
import numpy as np
import pandas as pd
import plotly.express as px
from tqdm import tqdm
import requests
pd.options.display.float_format = '{:.2f}'.format
import requests

In [206]:

# we use this data from each month
# url = "https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard/resolve/main/elo_results_20240305.pkl"
# url = "https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard/resolve/main/elo_results_20240403.pkl"
# url = "https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard/resolve/main/elo_results_20240501.pkl"
# url = "https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard/resolve/main/elo_results_20240602.pkl"
# url = "https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard/resolve/main/elo_results_20240706.pkl"
url = "https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard/resolve/main/elo_results_20240806.pkl"


response = requests.get(url)

with open("data.pkl", "wb") as file:
    file.write(response.content)

battle_info = pd.read_pickle("data.pkl")

battle_info.keys()

# battle_info["text"]["full"].keys()

dict_keys(['text', 'vision'])

In [207]:
battle_info["text"]["full"]["leaderboard_table_df"].sort_values("rating", ascending=False)
# battle_info["text"]["leaderboard_table_df"].sort_values("rating", ascending=False)
# battle_info["leaderboard_table_df"].sort_values("rating", ascending=False)

,rating,variance,rating_q975,rating_q025,num_battles,final_ranking
gemini-1.5-pro-exp-0801,1299.42,6.31,1303.42,1294.53,15244,1
gpt-4o-2024-05-13,1286.13,2.59,1288.68,1282.30,72589,2
gpt-4o-mini-2024-07-18,1276.81,6.33,1281.08,1271.42,16064,3
claude-3-5-sonnet-20240620,1271.24,3.18,1274.60,1267.63,42939,3
gemini-advanced-0514,1266.38,2.36,1269.36,1263.76,52126,4
...,...,...,...,...,...,...
chatglm-6b,878.85,25.51,886.34,868.44,4998,122
fastchat-t5-3b,868.24,27.25,878.12,858.73,4304,123
stablelm-tuned-alpha-7b,839.55,40.80,852.90,827.41,3333,125
dolly-v2-12b,822.15,30.87,832.61,812.79,3486,125


In [208]:

# Load the file
file_path = '/Users/buzz/Documents/Haim/Matrix-xx-csv.csv'

df = pd.read_csv(file_path)

# Drop any completely empty columns
df = df.dropna(axis=1, how='all')

# Initialize an empty list to store the combined results
result_list = []

# Get the unique models and sorted dates (months) in ascending order
models = df['model'].unique()
months = sorted(df['date'].unique(), key=lambda x: pd.to_datetime(x, format='%b-%y'))

# Iterate through each model and month to create the pivot
for model in models:
    model_df = df[df['model'] == model]
    model_data = {'model': model}
    
    for month in months:
        # Filter data by the model and month
        month_data = model_df[model_df['date'] == month]
        if not month_data.empty:
            # If there are multiple entries for the same model and month, concatenate them with commas or another delimiter
            model_data[month] = ', '.join(month_data['rating'].astype(str))
        else:
            model_data[month] = None
    
    # Append the model's data to the result list
    result_list.append(model_data)

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(result_list)

# Sort the DataFrame rows by the number of non-null values, in descending order
result_df['non_null_count'] = result_df.notnull().sum(axis=1) - 1  # Subtracting 1 to exclude the 'model' column
result_df = result_df.sort_values(by='non_null_count', ascending=False)

# Filter to keep only rows with 3 or more non-null values
result_df = result_df[result_df['non_null_count'] >= 4]

# Calculate the variation between the highest and lowest values in each row
def calculate_variation(row):
    numeric_values = []
    for value in row:
        try:
            numeric_values.append(float(value))
        except (ValueError, TypeError):
            continue
    if numeric_values:
        max_value = max(numeric_values)
        min_value = min(numeric_values)
        return ((max_value - min_value) / max_value -1 ) * 100
    return None

# Apply the variation calculation to each row
result_df['variation_percent'] = result_df.apply(calculate_variation, axis=1)

# Drop the 'non_null_count' column as it's no longer needed
result_df = result_df.drop(columns=['non_null_count'])

# Save the result DataFrame to a new CSV file
result_df.to_csv('Model_Month_Combined_Filtered_Variation.csv', index=False)


# 03/2024 min. 797.51 ; max. 1251.07
# 04/2024 min. 801.24 ; max. 1255.91
# 05/2024 min. 804.36 ; max. 1258.82
# 06/2024 min. 798.35 ; max. 1256.16
# 07/2024 min. 798.60 ; max. 1287.40
# 08/2024 min. 798.47 ; max. 1299.42


# Display the result DataFrame (in Jupyter, this will show a nicely formatted table)
result_df


,model,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,variation_percent
9,gpt-4-1106-preview,1251.07,1252.86,1252.68,1251.34,1250.94,1250.93,-0.48
13,gpt-4-0125-preview,1251.03,1249.15,1246.78,1245.91,1245.54,1245.4,-0.48
12,claude-3-opus-20240229,None,1255.91,1250.93,1248.28,1248.21,1247.87,-0.40
21,bard-jan-24-gemini-pro,1203.47,1206.21,1208.71,1208.37,1208.47,None,-0.41
28,gpt-4-0314,1185.99,1186.98,1189.56,1185.96,None,None,-0.34
7,gemini-1.5-pro-api-0409-preview,None,None,1249.62,1257.47,1257.51,1257.21,-0.32
8,gpt-4-turbo-2024-04-09,None,None,1258.82,1256.16,1257.03,1256.61,-0.32
25,claude-3-sonnet-20240229,None,1201.39,1201.27,1201.44,1200.96,None,-0.33


In [221]:
import pandas as pd

# Load the file
file_path = '/Users/buzz/Documents/Haim/Data-lmsys-csv.csv'
df = pd.read_csv(file_path)

# Drop any completely empty columns
df = df.dropna(axis=1, how='all')

# Initialize an empty list to store the combined results
result_list = []

# Get the unique models and sorted dates (months) in ascending order
models = df['model'].unique()
months = sorted(df['date'].unique(), key=lambda x: pd.to_datetime(x, format='%b-%y'))

# Normalization data
normalization_data_lmsys = {
    "Mar-24": {"min": 797.51, "max": 1251.07},
    "Apr-24": {"min": 801.24, "max": 1255.91},
    "May-24": {"min": 804.36, "max": 1258.82},
    "Jun-24": {"min": 798.35, "max": 1256.16},
    "Jul-24": {"min": 798.60, "max": 1287.40},
    "Aug-24": {"min": 798.47, "max": 1299.42}
}

# Normalization function
def normalize(value, min_val, max_val):
    return round((value - min_val) / (max_val - min_val), 2)

# Iterate through each model and month to create the pivot
for model in models:
    model_df = df[df['model'] == model]
    model_data = {'model': model}
    
    for month in months:
        # Filter data by the model and month
        month_data = model_df[model_df['date'] == month]
        if not month_data.empty:
            # Normalize the rating values and round to 2 decimal places
            month_str = pd.to_datetime(month, format='%b-%y').strftime('%b-%y')
            normalized_values = month_data['rating'].apply(
                lambda x: normalize(x, normalization_data_lmsys[month_str]["min"], normalization_data_lmsys[month_str]["max"])
            )
            # If there are multiple entries for the same model and month, concatenate them with commas or another delimiter
            model_data[month] = ', '.join(normalized_values.astype(str))
        else:
            model_data[month] = None
    
    # Append the model's data to the result list
    result_list.append(model_data)

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(result_list)

# Sort the DataFrame rows by the number of non-null values, in descending order
result_df['non_null_count'] = result_df.notnull().sum(axis=1) - 1  # Subtracting 1 to exclude the 'model' column
result_df = result_df.sort_values(by='non_null_count', ascending=False)

# Filter to keep only rows with 4 or more non-null values
result_df = result_df[result_df['non_null_count'] >= 2]

# Calculate the variation between the highest and lowest values in each row
def calculate_variation(row):
    numeric_values = []
    for value in row[1:]:  # Exclude the model column
        try:
            numeric_values.append(float(value))
        except (ValueError, TypeError):
            continue
    if numeric_values:
        max_value = max(numeric_values)
        min_value = min(numeric_values)
        return round(((max_value - min_value) / max_value -1) * 100, 2)
    return None

# Apply the variation calculation to each row
result_df['variation_percent'] = result_df.apply(calculate_variation, axis=1)

# Drop the 'non_null_count' column as it's no longer needed
result_df = result_df.drop(columns=['non_null_count'])

# Save the result DataFrame to a new CSV file
result_df.to_csv('Model_Month_Combined_Filtered_Normalized_Variation.csv', index=False)

# Display the result DataFrame
result_df


,model,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,variation_percent
9,gpt-4-1106-preview,1.0,0.99,0.99,0.99,0.93,0.9,-15.00
13,gpt-4-0125-preview,1.0,0.99,0.97,0.98,0.91,0.89,-14.83
12,claude-3-opus-20240229,None,1.0,0.98,0.98,0.92,0.9,-18.00
21,bard-jan-24-gemini-pro,0.9,0.89,0.89,0.9,0.84,None,-16.80
28,gpt-4-0314,0.86,0.85,0.85,0.85,None,None,-21.25
7,gemini-1.5-pro-api-0409-preview,None,None,0.98,1.0,0.94,0.92,-23.00
8,gpt-4-turbo-2024-04-09,None,None,1.0,1.0,0.94,0.91,-22.75
25,claude-3-sonnet-20240229,None,0.88,0.87,0.88,0.82,None,-20.50
27,command-r-plus,None,None,0.85,0.85,0.8,None,-26.67
37,qwen1.5-72b-chat,0.77,0.77,0.77,None,None,None,-25.67


In [ ]:
normalization_data_helm = {
    "Mar-24": {"min": 0.045, "max": 0.965},
    "Apr-24": {"min": 0.05, "max": 0.958},
    "May-24": {"min": 0.072, "max": 0.957},
    "Jun-24": {"min": 0.064, "max": 0.973},
    "Jul-24": {"min": 0.06, "max": 0.965},
    "Aug-24": {"min": 0.052, "max": 0.963}

In [220]:
import pandas as pd

# Load the file
file_path = '/Users/buzz/Documents/Haim/Data-helm-lite.csv'
df = pd.read_csv(file_path)

# Drop any completely empty columns
df = df.dropna(axis=1, how='all')

# Initialize an empty list to store the combined results
result_list = []

# Get the unique models and sorted dates (months) in ascending order
models = df['model'].unique()
months = sorted(df['date'].unique(), key=lambda x: pd.to_datetime(x, format='%b-%y'))

# Normalization data
normalization_data_helm = {
    "Mar-24": {"min": 0.045, "max": 0.965},
    "Apr-24": {"min": 0.05, "max": 0.958},
    "May-24": {"min": 0.072, "max": 0.957},
    "Jun-24": {"min": 0.064, "max": 0.973},
    "Jul-24": {"min": 0.06, "max": 0.965},
    "Aug-24": {"min": 0.052, "max": 0.963}
}


# Normalization function
def normalize(value, min_val, max_val):
    return round((value - min_val) / (max_val - min_val), 2)

# Iterate through each model and month to create the pivot
for model in models:
    model_df = df[df['model'] == model]
    model_data = {'model': model}
    
    for month in months:
        # Filter data by the model and month
        month_data = model_df[model_df['date'] == month]
        if not month_data.empty:
            # Normalize the rating values and round to 2 decimal places
            month_str = pd.to_datetime(month, format='%b-%y').strftime('%b-%y')
            normalized_values = month_data['rating'].apply(
                lambda x: normalize(x, normalization_data_helm[month_str]["min"], normalization_data_helm[month_str]["max"])
            )
            # If there are multiple entries for the same model and month, concatenate them with commas or another delimiter
            model_data[month] = ', '.join(normalized_values.astype(str))
        else:
            model_data[month] = None
    
    # Append the model's data to the result list
    result_list.append(model_data)

# Convert the list of dictionaries to a DataFrame
result_df_helm_lite = pd.DataFrame(result_list)

# Sort the DataFrame rows by the number of non-null values, in descending order
result_df_helm_lite['non_null_count'] = result_df_helm_lite.notnull().sum(axis=1) - 1  # Subtracting 1 to exclude the 'model' column
result_df_helm_lite = result_df_helm_lite.sort_values(by='non_null_count', ascending=False)

# Filter to keep only rows with 4 or more non-null values
result_df_helm_lite = result_df_helm_lite[result_df_helm_lite['non_null_count'] >= 2]

# Calculate the variation between the highest and lowest values in each row
def calculate_variation(row):
    numeric_values = []
    for value in row[1:]:  # Exclude the model column
        try:
            numeric_values.append(float(value))
        except (ValueError, TypeError):
            continue
    if numeric_values:
        max_value = max(numeric_values)
        min_value = min(numeric_values)
        return round(((max_value - min_value) / max_value -1) * 100, 2)
    return None

# Apply the variation calculation to each row
result_df_helm_lite['variation_percent'] = result_df_helm_lite.apply(calculate_variation, axis=1)

# Drop the 'non_null_count' column as it's no longer needed
result_df_helm_lite = result_df_helm_lite.drop(columns=['non_null_count'])

# Save the result DataFrame to a new CSV file
result_df_helm_lite.to_csv('Model_Month_Combined_Filtered_Normalized_Variation_Helm_Lite.csv', index=False)

# Display the result DataFrame
result_df_helm_lite


,model,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,variation_percent
13,Palmyra X V3 (72B),0.69,0.7,0.69,0.68,0.69,0.69,-11.33
12,GPT-4 Turbo (1106 preview),0.84,0.84,0.84,0.83,0.84,0.84,-13.83
2,GPT-4 (0613),0.84,0.84,0.84,0.83,0.83,0.84,-13.83
19,Palmyra X V2 (33B),0.6,0.6,0.59,0.59,0.59,0.6,-9.83
17,PaLM-2 (Unicorn),0.69,0.7,0.69,0.68,0.69,0.69,-11.33
24,Mixtral (8x7B 32K seqlen),0.66,0.66,0.66,0.65,0.65,None,-13.00
20,Yi (34B),0.66,0.67,0.66,0.65,0.66,None,-13.00
6,Llama 3 (70B),None,0.8,0.8,0.78,0.79,0.8,-15.60
18,Qwen1.5 (72B),None,0.68,0.68,0.67,0.67,0.68,-13.40
11,Mixtral (8x22B),None,0.72,0.71,0.7,0.71,0.72,-14.00


In [222]:
import pandas as pd

# Load the file
file_path = '/Users/buzz/Documents/Haim/Data-lmsys-csv.csv'
df = pd.read_csv(file_path)

# Drop any completely empty columns
df = df.dropna(axis=1, how='all')

# Initialize an empty list to store the combined results
result_list = []

# Get the unique models and sorted dates (months) in ascending order
models = df['model'].unique()
months = sorted(df['date'].unique(), key=lambda x: pd.to_datetime(x, format='%b-%y'))

# Normalization data
normalization_data_lmsys = {
    "Mar-24": {"min": 797.51, "max": 1251.07},
    "Apr-24": {"min": 801.24, "max": 1255.91},
    "May-24": {"min": 804.36, "max": 1258.82},
    "Jun-24": {"min": 798.35, "max": 1256.16},
    "Jul-24": {"min": 798.60, "max": 1287.40},
    "Aug-24": {"min": 798.47, "max": 1299.42}
}

# Normalization function
def normalize(value, min_val, max_val):
    return round((value - min_val) / (max_val - min_val), 2)

# Iterate through each model and month to create the pivot
for model in models:
    model_df = df[df['model'] == model]
    model_data = {'model': model}
    
    for month in months:
        # Filter data by the model and month
        month_data = model_df[model_df['date'] == month]
        if not month_data.empty:
            # Normalize the rating values and round to 2 decimal places
            month_str = pd.to_datetime(month, format='%b-%y').strftime('%b-%y')
            normalized_values = month_data['rating'].apply(
                lambda x: normalize(x, normalization_data_lmsys[month_str]["min"], normalization_data_lmsys[month_str]["max"])
            )
            # If there are multiple entries for the same model and month, concatenate them with commas or another delimiter
            model_data[month] = ', '.join(normalized_values.astype(str))
        else:
            model_data[month] = None
    
    # Append the model's data to the result list
    result_list.append(model_data)

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(result_list)

# Sort the DataFrame rows by the number of non-null values, in descending order
result_df['non_null_count'] = result_df.notnull().sum(axis=1) - 1  # Subtracting 1 to exclude the 'model' column
result_df = result_df.sort_values(by='non_null_count', ascending=False)

# Filter to keep only rows with 4 or more non-null values
result_df = result_df[result_df['non_null_count'] >= 2]

# Calculate the variation between the months for each model
def calculate_variation(row):
    numeric_values = []
    for month in months:  # Iterate over the month columns
        value = row.get(month)
        try:
            numeric_values.append(float(value))
        except (ValueError, TypeError):
            continue
    if len(numeric_values) > 1:
        max_value = max(numeric_values)
        min_value = min(numeric_values)
        return round(((max_value - min_value) / max_value) * 100, 2)
    return None

# Apply the variation calculation to each row
result_df['variation_percent'] = result_df.apply(calculate_variation, axis=1)

# Drop the 'non_null_count' column as it's no longer needed
result_df = result_df.drop(columns=['non_null_count'])

# Save the result DataFrame to a new CSV file
result_df.to_csv('Model_Month_Combined_Filtered_Normalized_Variation.csv', index=False)

# Display the result DataFrame
result_df


,model,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,variation_percent
9,gpt-4-1106-preview,1.0,0.99,0.99,0.99,0.93,0.9,10.00
13,gpt-4-0125-preview,1.0,0.99,0.97,0.98,0.91,0.89,11.00
12,claude-3-opus-20240229,None,1.0,0.98,0.98,0.92,0.9,10.00
21,bard-jan-24-gemini-pro,0.9,0.89,0.89,0.9,0.84,None,6.67
28,gpt-4-0314,0.86,0.85,0.85,0.85,None,None,1.16
7,gemini-1.5-pro-api-0409-preview,None,None,0.98,1.0,0.94,0.92,8.00
8,gpt-4-turbo-2024-04-09,None,None,1.0,1.0,0.94,0.91,9.00
25,claude-3-sonnet-20240229,None,0.88,0.87,0.88,0.82,None,6.82
27,command-r-plus,None,None,0.85,0.85,0.8,None,5.88
37,qwen1.5-72b-chat,0.77,0.77,0.77,None,None,None,0.00
